In [2]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from hierarqcal import (
    Qhierarchy,
    Qcycle,
    Qpermute,
    Qmask,
    Qunmask,
    Qpivot,
    Qinit,
    Qmotif,
    Qmotifs,
    plot_motif,
    plot_circuit,
    Qunitary,
)
from pennylane import AngleEmbedding
import pennylane as qml
from hierarqcal.pennylane.pennylane_circuits import V2, U2, V4


In [8]:
def ansatz_pool_2(bits, symbols=None):
    qml.CNOT(wires=[bits[0], bits[1]])
U_ansatz_pool_2 = Qunitary(ansatz_pool_2, n_symbols=0, arity=2)


In [34]:
mask = Qinit(8) + Qmask("10", mapping=U_ansatz_pool_2, strides=0)
plot_motif(mask[1])
plot_circuit(mask)

TypeError: Qmotif.__init__() got an unexpected keyword argument 'step'